In [1]:
import re
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from nltk.corpus import stopwords

## Chargement et préprosessing des données

In [2]:
df=pd.read_csv('X_train_update.csv')
df1=pd.read_csv("y_train_CVw08PX.csv")

In [3]:

# Codage de la colone prdttype de 1 à 27
l=[]
for mm in df1.prdtypecode:
    if mm not in l :
        l.append(mm)
l.sort()
for i, j in enumerate(l) :
    for mmm in df1.prdtypecode :
        if mmm==j:
            df1[df1.prdtypecode==mmm]=i+1

# Regroupement de la variable description et designation             
df["description"] = (df["designation"].fillna('') + " " + df["description"].fillna('')).str.lower()
df["categ"] = df1.prdtypecode

# Instanciation de stopwords 
stop_words = set([
    "le", "la", "les", "de", "du", "des", "et", "en", "un", "une", "pour", 
    "avec", "dans", "par", "sur", "au", "aux", "ce", "ces", "se", "ses", 
    "est", "qui", "que", "dont", "ou", "à", "son", "sa", "leurs", "comme"
])
stop_words_fr = set(stopwords.words("french"))
stop_words_en = set(stopwords.words("english"))
stop_words_de = set(stopwords.words("german"))
stop_words_es = set(stopwords.words("spanish"))
stop_words_it = set(stopwords.words("italian"))
stop_words_pt = set(stopwords.words("portuguese"))
stop_words_misc = set(["br", "li", "x", "p", "ul", "div", "strong", "nbsp", "eacute", "amp", ])  # Mots divers à ignorer

# Fusionner les stopwords
stop_words_all = stop_words_fr | stop_words_en | stop_words_de | stop_words_es | stop_words_it | stop_words_pt | stop_words_misc | stop_words

##
def nettoyer_texte(texte):
    texte = str(texte).lower()  
    texte = re.sub(r'\d+', '', texte)  
    texte = re.sub(r'\W+', ' ', texte) 
    texte = ' '.join([mot for mot in texte.split() if mot not in stop_words_all])  
    return texte
##     
df['description'] = df['description'].apply(nettoyer_texte)

## Séparation des données en train et test

In [21]:
# Séparons les données en un ensemble d'apprentissage et un ensemble de test (20%), 
#avec X comme les variables explicatives et y comme la variable cible.
X=df.description
y=df.categ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [22]:
# Vectorisation 
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Le modèle

In [23]:

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(X_train, y_train)


In [ ]:
# Prédictions
y_pred = clf.predict(X_test)

In [12]:
# Calcul du F1-score moyen (macro, micro ou weighted selon nos besoin)
for f1_sc in ["macro", "micro", "weighted"]:
    f1 = f1_score(y_test, y_pred, average=f1_sc)  
    print(f"F1-score moyen {f1_sc}:", f1)

F1-score moyen macro: 0.4661444304934998
F1-score moyen micro: 0.45843146490814884
F1-score moyen weighted: 0.5014781404127734


In [14]:
# Classification_report et la matrice de confusion 
print(classification_report(y_test, y_pred))
mconfusion=pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
mconfusion

              precision    recall  f1-score   support

           1       0.29      0.19      0.23       622
           2       0.51      0.31      0.39       521
           3       0.55      0.25      0.34       355
           4       0.78      0.65      0.71       190
           5       0.73      0.34      0.47       572
           6       0.92      0.31      0.46       762
           7       0.69      0.32      0.44       162
           8       0.59      0.20      0.30       993
           9       0.28      0.23      0.25       398
          10       0.86      0.79      0.82      1036
          11       0.94      0.69      0.80       143
          12       0.95      0.43      0.59       474
          13       0.69      0.49      0.57       633
          14       0.72      0.42      0.53      1028
          15       0.87      0.67      0.76       852
          16       0.70      0.12      0.21       173
          17       0.64      0.38      0.48       977
          18       0.00    

C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Classe prédite,1,2,3,4,5,6,7,8,9,10,...,17,19,20,21,22,23,24,25,26,27
Classe réelle,,,,,,,,,,,,,,,,,,,,,
1,116,0,0,2,0,0,0,0,0,0,...,0,500,1,0,2,0,0,0,1,0
2,30,164,20,0,2,0,0,2,1,0,...,0,281,0,16,0,1,0,0,1,2
3,3,0,88,1,7,0,0,0,1,7,...,1,159,4,66,5,1,2,0,3,1
4,2,0,13,124,0,0,0,1,0,0,...,0,33,1,12,1,0,0,0,0,1
5,8,3,1,1,196,1,7,3,1,0,...,3,332,9,0,4,0,0,0,2,0
6,13,0,0,1,1,236,0,1,6,2,...,0,482,11,0,1,0,0,0,0,0
7,4,0,0,1,14,0,52,2,0,0,...,1,80,5,0,1,0,0,0,1,0
8,8,17,1,0,30,1,2,203,124,113,...,12,416,16,1,1,8,2,0,0,0
9,7,0,1,3,6,2,9,33,91,5,...,1,222,4,0,2,1,4,0,0,0


In [18]:
# Les colones non affichées ci-dessous sauf celle qui correspond à la type non predit par le modèle.
mconfusion[[11, 12, 13, 14, 15, 16, 17, 19,20]]

Classe prédite,11,12,13,14,15,16,17,19,20
Classe réelle,,,,,,,,,
1,0,0,0,0,0,0,0,500,1
2,1,0,0,0,0,0,0,281,0
3,0,0,4,2,0,0,1,159,4
4,0,1,1,0,0,0,0,33,1
5,0,1,0,0,0,0,3,332,9
6,0,1,7,0,0,0,0,482,11
7,0,0,1,0,0,0,1,80,5
8,0,4,26,7,1,0,12,416,16
9,1,0,6,0,0,0,1,222,4
